In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}


result = ''
for page in range(1, 5):
    r = requests.get(f'https://novosibirsk.hh.ru/search/vacancy?page={page}', headers=headers)
    result+=r.content.decode()

soup=bs(result, 'html.parser')
vacancies = soup.find_all('div', {'class': 'vacancy-serp-item-body'})


entries = []
for vacancy in vacancies:
    
    title = (vacancy.find("a").text)
    try:
        salary = (vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).contents)
        currency = salary[-1]
        if 'от' in salary[0]:
            salary_min = int(salary[2].replace('\u202f',''))
            salary_max = '-'
        else:
            salary_min, salary_max = map(int, salary[0].replace('\u202f','').split(' – '));
    except:
        salary_min = salary_max = currency =  ('-')
    link = (vacancy.find('a')['href'].split('?')[0])
    entry = [title, salary_min, salary_max, currency, link]
    entries.append(entry)


df = pd.DataFrame(entries, columns=['title', 'salary_min', 'salary_max', 'currency', 'link'])
df.head()


,title,salary_min,salary_max,currency,link
0,Junior Manual QA,-,-,-,https://novosibirsk.hh.ru/vacancy/53731698
1,Официант в КМ20,80000,115000,руб.,https://novosibirsk.hh.ru/vacancy/55167316
2,Ассистент рекрутера / HR,40000,70000,руб.,https://novosibirsk.hh.ru/vacancy/55171864
3,Менеджер по продажам (без поиска),90000,153000,руб.,https://novosibirsk.hh.ru/vacancy/55256397
4,Специалист по выдаче заказов (Витебск),780,1260,бел. руб.,https://novosibirsk.hh.ru/vacancy/53547175
